In [ ]:
# 有道词典 definitions 
defs = driver.find_element_by_xpath('//*[@id="phrsListTab"]/div[2]/ul/p').text
# LINE Dict. definitions
defs = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[3]/ol').text

In [1]:
# Import necessary modules
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
from io import StringIO
import json
import urllib.request
import pandas as pd
import numpy as np
import playsound
from numpy import random
from datetime import datetime
from langdetect import detect
import winsound
from playsound import playsound
from gtts import gTTS
import os
from datetime import timedelta

In [2]:
# Do I read or listen today for new vocab?
file = open('what_do.txt', mode = 'r')
what_do = file.read()
file.close()
if what_do == 'listening':
    print("Listening Today")
    f = open('what_do.txt', 'w+')
    f.write('reading')
    f.close()
elif what_do == 'reading':
    print("Reading Today")
    f = open('what_do.txt', 'w+')
    f.write('listening')
    f.close()
else:
    pass

Reading Today


In [3]:
# What words do I write sentences for today?
languages = ['中文', '한국어', '日本語']
sentenceDate = datetime.now().date()
for i in languages:
    # Read the Excel files into DataFrames
    wordsDF = pd.read_excel(r'Databases\{}_words.xlsx'.format(i), sheet_name = 'words', header = 0)
    sentencesDF =  pd.read_excel(r'Databases\{}_sentences.xlsx'.format(i), sheet_name = 'sentences', header = 0)
    # Read the Chinese Databases into DataFrames
    words = pd.read_excel(r'Databases\{}_words.xlsx'.format(i), sheet_name = 'words', header = 0)
    # Define the date that we want to retreive
    desired_date = (sentenceDate - timedelta(days = 14)).strftime("%Y-%m-%d")
    # Retrieve the words from the specified date
    write = wordsDF['FOREIGN'].where(words['DATE'] == desired_date)
    # Drop NAN rows
    write = write.dropna().reset_index(drop = True)
    # Display the words with which to write sentences
    print("--------{}--------\n{}\n".format(i, write))

--------中文--------
Series([], Name: FOREIGN, dtype: object)

--------한국어--------
Series([], Name: FOREIGN, dtype: object)

--------日本語--------
Series([], Name: FOREIGN, dtype: object)



In [4]:
# Define the current date and time
now = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
startTime1 = time.time()
# Define the three functions we'll be using
# Define Vocab Notes function
def addVocab(language):
    n = 0
    # Handle the unique 日本語
    if language == '日本語':
    # Loop through all the rows and create a note for each one
        for index, vocab in enumerate(vocabForeign):
            try:
                invoke('addNote', note = {
                            "deckName": '{}'.format(language),
                            "modelName": '(NEW) {}'.format(language),
                            "fields": {
                                "(Word) Foreign": vocab, 
                                "(Word) English": vocabEnglish[index],
                                "(Word) Foreign [漢字]": kanji[index]
                                }  
                    })
            except Exception:
                continue
            n += 1
            print("{} has elapsed for {} Vocab words of {} ({} remaining).".format(convert(time.time() - startTime1), n, words.shape[0], words.shape[0] - n))
    # For 한국어 and 中文
    else:
        # Loop through all the rows and create a note for each one
        for index, vocab in enumerate(vocabForeign):
            try:
                invoke('addNote', note = {
                            "deckName": '{}'.format(language),
                            "modelName": '(NEW) {}'.format(language),
                            "fields": {
                                "(Word) Foreign": vocab, 
                                "(Word) English": vocabEnglish[index]
                                }  
                    })
            except Exception:
                continue
            n += 1
            print("{} has elapsed for {} Vocab words of {} ({} remaining).".format(convert(time.time() - startTime1), n, words.shape[0], words.shape[0] - n))
# Define Cloze Notes function
def addCloze(language):
    n = 0
    # Loop through all the rows and create a note for each one
    for index, sentence in enumerate(cloze):
        try:
            invoke('addNote', note = {
                        "deckName": '{}'.format(language),
                        "modelName": '(NEW) {} Cloze'.format(language),
                        "fields": {
                            "Text": sentence, 
                            "Tip": tip[index],
                            "English": english[index],
                            "full setence": fullSentence[index]
                            }  
                })
        except Exception:
            continue
        n += 1
        print("{} has elapsed for {} Cloze notes of {} ({} remaining).".format(convert(time.time() - startTime1), n, dfm.shape[0], dfm.shape[0] - n))

# Define the Sentence Scraping function

def scrapeSentences(language):
    words = vocab['Foreign']
    n = 0
    o = words.shape[0]
    # Create master DataFrame
    w = {}
    concatList = []
    # Run the scraper based on the language
    if language == '日本語':
        # Not done
        # See CSV_Gen [ALL].ipynb
        pass
    elif language == '中文':
        # Define URL for Chinese words
        url_zh = 'https://dict.naver.com/linedict/zhendict/dict.html#/cnen/home'
        # Define the driver for Selenium to use
        driver = webdriver.Chrome('chromedriver.exe')
        # Open the specfied URL
        driver.get(url_zh)
        # Find the radio group and select "Examples"
        examples_radio = driver.find_element_by_xpath('//*[@id="ac_form"]/fieldset/ul/li[2]/span').click()
        # Find the Search bar
        search = driver.find_element_by_xpath('//*[@id="ac_input"]')
        # Scrape sentences for each word present in the vocabulary list
        for word in words:
            # Type the word into the Search Box
            search.send_keys(word)
            search.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(3)
            # Find the sentences examples
            examples = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]')
            if examples is not None:
                # Sleep to avoid errors
                time.sleep(1)
                # Define an object with all the example sentences
                unfiltered_sentences = examples.text
                # Clean the string of text that was just scraped
                pass1 = unfiltered_sentences.replace("more", "")
                # Finalize the DataFrame
                myList = pass1.split("\n")
                myList = myList[::2]
                # Drop the 拼音 be creating a dictionary
                myDict = dict(zip(myList[::2], myList[1::2]))
                foreign = myDict.keys()
                english = myDict.values()
                df = pd.DataFrame(columns = ['English', 'Foreign', 'Word', 'Hint', 'Cloze'])
                df['English'] = english
                df['Foreign'] = foreign
                df['Word'] = word
                wordsClean = words.tolist()
                wordsClean = str(wordsClean).strip("[]")
                wordsClean = wordsClean.replace("'", "")
                df['Hint'] = wordsClean
                df['Hint'] = df['Hint'].str.strip('[]')
                df['Hint'] = df['Hint'].str.replace("'", "")
                # Create the Cloze sentences
                df['Cloze'] = df['Foreign'].str.replace(word, '{{c1::' + word + '}}')
                # Remove examples that don't contain a period
                df['English'] = df['English'].map(str)
                df1 = df[df['English'].str.contains('\.', na = False)]
                # Reduce the DataFrame to only 3 sentences
                df2 = df1.iloc[:3]
                # Create the final DataFrame for the word
                w[word] = df2
                # Add the DataFrame to the list of all word DataFrames
                concatList.append(w[word])
                # Sleep to avoid errors
                time.sleep(0.5)
            else:
                pass
            n += 1
            o -= 1
            print("Elapsed time: ({}) {} for {} words ({} remaining)".format(word, convert(time.time() - startTime1), n, o))
            # Exit the FOR loop
        # Quit the browser
        driver.quit()
        # Concatenate each word's DataFrame to the final one with all the sentences
        dfm = pd.concat(concatList, ignore_index = True)
        # Fix formating issues
        dfm['Cloze'] = dfm['Cloze'].str.replace('Name:.*', '')
        dfm['Cloze'] = dfm['Cloze'].str.replace('\n', ',')
        #dfm['Cloze'] = dfm['Cloze'].str.replace('\d', '')
        # return the created DataFrame
        return dfm
    elif language == '한국어':
        # Define URL for Korean words
        url_ko = 'https://en.dict.naver.com/#/search?range=example&shouldSearchVlive=false&query=%ED%9B%84%ED%9A%8C&exampleLevel=exist:1&haveTrans=exist:1'
        # Define the driver for Selenium to use
        driver = webdriver.Chrome('chromedriver.exe')
        # Open the specfied URL
        driver.get(url_ko)
        # Scrape sentences for each word present in the vocabulary list
        for word in words:
            # Sleep to avoid errors
            time.sleep(2)
            # Find the Search bar
            search = driver.find_element_by_xpath('//*[@id="ac_input"]')
            # Sleep to avoid errors
            time.sleep(1)
            # Type the word into the Search Box
            search.clear()
            # Sleep to avoid errors
            time.sleep(1)
            search.send_keys(word)
            search.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(3)
            # Find the button titled "Examples" and click it
            example_button = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[1]/div/div/a[4]')
            example_button.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(1)
            # Sleep to avoid errors
            time.sleep(2)
            # Find the "Have Translations" check box and check it
            have_translations = driver.find_element_by_xpath('//*[@id="searchPage_example"]/div[1]/div/span[5]/label').click()
            # Sleep to avoid errors
            time.sleep(2.5)
            # Sleep to avoid errors
            time.sleep(2)
            # Find the sentence examples
            examples = driver.find_element_by_xpath('//*[@id="searchPage_example"]/div[2]')
            # Define an object with all the example sentences
            unfiltered_sentences = examples.text
            pass1 = unfiltered_sentences.replace("발음듣기", "")
            pass2 = pass1.replace("Oxford Advanced Learner's English-Korean Dictionary", "")
            pass3 = pass2.replace("절대어휘 5100", "")
            pass4 = pass3.replace("국제영어대학원대학교 신어사전", "")
            pass5 = pass4.replace("DARAKWON", "")
            pass6 = pass5.replace("Friendict Level English-Korean Dictionary", "")
            pass7 = pass6.replace("Web Collection", "")
            pass8 = pass7.replace("YBM All in All English-Korean Dictionary", "")
            pass9 = pass8.replace("Neungyule", "")
            pass10 = pass9.replace("User translation", "")
            pass11 = pass10.replace("Hansard", "")
            pass12 = pass11.replace("영어영작문대사전", "")
            pass13 = pass12.replace("English Hidden Card", "")
            pass14 = pass13.replace("TIMES CORE", "")
            pass15 = pass14.replace("Dong-a's Prime English-Korean Dictionary", "")
            # Create a Dictionary with which to create the final DataFrame
            myList = pass15.split("\n\n")
            # Remove the parentheitcal words from the list
            myList2 = []
            for i in myList:
                i = re.sub(r'\([^()]*\)', '', i)
                i = i.lstrip()
                i = i.rstrip("\n")
                myList2.append(i)
            # Make dictionary from the new list
            myDict = dict(zip(myList2[::2], myList2[1::2]))
            english = myDict.keys()
            foreign = myDict.values()
            # Create a DataFrame from the scraped string
            df = pd.DataFrame(columns = ['English', 'Foreign', 'Word', 'Hint', 'Cloze'])
            df['English'] = english
            df['Foreign'] = foreign
            df['Word'] = word
            wordsClean = words.tolist()
            wordsClean = str(wordsClean).strip("[]")
            wordsClean = wordsClean.replace("'", "")
            df['Hint'] = wordsClean
            df['Hint'] = df['Hint'].str.replace("'", "")
            # Handle conjugations
            if "하다" in word:
                wordConj = word[:-2]
            elif "다" in word:
                wordConj = word[:-1]
            else:
                wordConj = word
            # Create the Cloze sentences    
            df['Cloze'] = df['Foreign'].str.replace(wordConj, '{{c1::' + wordConj + '}}')
            # Remove examples that don't contain a period
            df['English'] = df['English'].map(str)
            # Reduce the DataFrame to only 3 sentences
            df2 = df.iloc[:3]
            # Create the final DataFrame for the word
            w[word] = df2
            # Add the DataFrame to the list of all word DataFrames
            concatList.append(w[word])
            n += 1
            o -= 1
            print("Elapsed time: ({}) {} for {} words ({} remaining)".format(word, convert(time.time() - startTime1), n, o))
            # Exit the FOR loop
        # Quit the browser
        driver.quit()
        # Concatenate each word's DataFrame to the final one with all the sentences
        dfm = pd.concat(concatList, ignore_index = True)
        # Remove the pesky fuckers that got through the cleaning passes filters
        dfm['Foreign'] = dfm['Foreign'].str.replace('\n.*', '')
        dfm['Cloze'] = dfm['Cloze'].str.replace('\n.*', '')
        # return the created DataFrame
        return dfm
        # Exit the IF-ELIF statement
    elif language == '日本語':
        pass

# DEFINE FUNCTIONS

def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%dh %02dm %02ds" % (hour, minutes, seconds)

def convertTTS(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%d hours %02d minutes %02d seconds" % (hour, minutes, seconds)

# Define the Anki-Connect functions
def request(action, **params):
    return {'action': action, 'params': params, 'version': 6}
def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']

# Define the Database Updating function
def updateDatabase(language):
    # Load the current Database into DataFrames
    wordsDF = pd.read_excel(r'Databases\{}_words.xlsx'.format(language), sheet_name = 'words', header = 0)
    sentencesDF =  pd.read_excel(r'Databases\{}_sentences.xlsx'.format(language), sheet_name = 'sentences', header = 0)
    # Get today's date and store it as a string
    date = datetime.now().strftime("%Y-%m-%d")
    if language != '日本語':  
        # Format the DataFrame to be appended to our local Database
        dfmAppend = dfm.drop(axis = 1, columns = ['Hint', 'Cloze'])
        # Name the columns in accordance with Flashcards.xlsx
        dfmAppend.columns = ['ENGLISH', 'FOREIGN', 'WORD']
        # Set the date of each row to today's date
        dfmAppend['DATE'] = date
    elif language == '日本語':
        pass
    # Create the Vocabulary DataFrame that will be appended 
    if language != '日本語':
        vocabAppend = vocab
        vocabAppend.columns = ['FOREIGN', 'ENGLISH']
    elif language == '日本語':
        vocabAppend = vocab
        vocabAppend.columns = ['FOREIGN', 'ENGLISH', 'KANJI']
    vocabAppend['DATE'] = date

    # Loop through the vocabAppend DF to find duplicates
    for index, row in vocabAppend.iterrows():
        word = vocabAppend['FOREIGN'][index]
        if wordsDF['FOREIGN'].str.contains(word).sum() > 0:
            vocabAppend.drop(axis = 0, index = index, inplace = True)
        
    # Loop through the dfmAppend DF to find duplicates
    for index, row in dfmAppend.iterrows():
        word = dfmAppend['WORD'][index]
        if wordsDF['FOREIGN'].str.contains(word).sum() > 0:
            dfmAppend.drop(axis = 0, index = index, inplace = True)
    # Append the new rows to the DataFrames
    wordsDF = wordsDF.append(vocabAppend)

    # Create new XLSX files including the appended rows
    wordsDF.to_excel('Databases\{}_words.xlsx'.format(language), sheet_name = 'words', index = False)

    if language != '日本語':
        sentencesDF = sentencesDF.append(dfmAppend)
        sentencesDF.to_excel('Databases\{}_sentences.xlsx'.format(language), sheet_name = 'sentences', index = False)
    elif language == '日本語':
        pass

# THE REAL WORK BEGINS

# Read CSVs into DataFrames
vocab = pd.read_excel(r'Flashcards.xlsx', sheet_name = 'vocab', header = 0)
vocab.columns = ['Foreign', 'English', 'Kanji', 'Language']
vocab['English'] = vocab['English'].replace('s/\([^)]*\)//', '')
# Retrieve the language from the vocab DataFrame then drop its column
language = vocab['Language'].loc[0:0].to_string(index = False, header = False)
vocab.drop(['Language'], inplace = True, axis = 1)
# Strip the leading space from the lang variable (Thank you shit Excel dropdown menu! /s)
language = language.lstrip()
# Drop nan values in the Vocab DataFrame
if language == '日本語':
    vocab.dropna(axis = 0, inplace = True)
elif language == '中文' or '한국어': 
    vocab.drop(['Kanji'], inplace = True, axis = 1)
    vocab.dropna(axis = 0, inplace = True)

# Define the returned DataFrame from scrapeSentences()
words = vocab['Foreign']
print("Processing {} {} words for scraping...".format(words.shape[0], language))
dfm = scrapeSentences(language)
# Define the necessary variables
vocabForeign = vocab['Foreign'].to_list()
vocabEnglish = vocab['English'].to_list()
# Define Kanji if it exists
if 'Kanji' in vocab.columns:
    kanji = vocab['Kanji'].to_list()
else:
    pass
# Add the Notes
print("Processing {} {} Vocabulary...".format(words.shape[0], language))
addVocab(language)
# Define the necessary variables
if language != '日本語':
    cloze = dfm['Cloze'].to_list()
    tip = dfm['Hint'].to_list()
    english = dfm['English'].to_list()
    fullSentence = dfm['Foreign'].to_list()
    print("Processing {} {} Clozes...".format(dfm.shape[0], language))
    addCloze(language)
print("Updating {} Databases...".format(language))
updateDatabase(language)
print("ALL PROCESSES COMPLETED! ELAPSED TIME: {}".format(convert(time.time() - startTime1)))
tts = gTTS(text = "All processes have completed in {}.".format(convertTTS(time.time() - startTime1)), lang = "en")
tts.save("tts.mp3")
playsound("tts.mp3")
os.remove("tts.mp3")

Processing 11 中文 words for scraping...
Elapsed time: (奉行) 0h 00m 09s for 1 words (10 remaining)
Elapsed time: (吝啬) 0h 00m 14s for 2 words (9 remaining)
Elapsed time: (泵) 0h 00m 19s for 3 words (8 remaining)
Elapsed time: (引擎) 0h 00m 24s for 4 words (7 remaining)
Elapsed time: (往复) 0h 00m 29s for 5 words (6 remaining)
Elapsed time: (螺旋桨) 0h 00m 33s for 6 words (5 remaining)
Elapsed time: (胡言乱语) 0h 00m 38s for 7 words (4 remaining)
Elapsed time: (谣言) 0h 00m 43s for 8 words (3 remaining)
Elapsed time: (暴力) 0h 00m 48s for 9 words (2 remaining)
Elapsed time: (蔓延) 0h 00m 53s for 10 words (1 remaining)
Elapsed time: (遗漏) 0h 00m 58s for 11 words (0 remaining)
Processing 11 中文 Vocabulary...
0h 01m 02s has elapsed for 1 Vocab words of 11 (10 remaining).
0h 01m 04s has elapsed for 2 Vocab words of 11 (9 remaining).
0h 01m 07s has elapsed for 3 Vocab words of 11 (8 remaining).
0h 01m 09s has elapsed for 4 Vocab words of 11 (7 remaining).
0h 01m 12s has elapsed for 5 Vocab words of 11 (6 remaining)

In [ ]:
updateDatabase(language)

In [ ]:
sent = pd.read_excel(r'Databases\{}_sentences.xlsx'.format(language), sheet_name = 'sentences', header = 0)
start = 236
for index, row in sent.iterrows():
    sent['CLOZE'][index] = sent['FOREIGN'][index].replace(sent['WORD'][index], '{{c1::' + sent['WORD'][index] + '}}')
sentences = sent['CLOZE']
english = sent['ENGLISH'][start:].to_list()
tip = sent['WORD'][start:].to_list()

In [ ]:
addCloze(language)